## Esofagus/Hypofarynx

In [1]:
# Importera bibliotek
import matplotlib.pyplot as plt # plots
import numpy as np # math
import pandas as pd
pd.options.display.width = 0
import plotly.offline as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px
#from scipy.interpolate import interp1d #används ej i denna kod
import IPython
py.offline.init_notebook_mode(connected=True)

In [2]:
# Importera data
data_path = 'CG_data/EsofagusHypofarynx2020.json'

data = pd.read_json(data_path)

# Översättningstabell
names_data_path = 'CG_data/Operatörsöversättningslista.csv'

names = pd.read_csv(names_data_path)

In [3]:
#Kontrollera vad kolumnerna heter
#print(data.columns)
#print(names.columns)

#Ändra kolumn för att kunna göra en join
names.columns = ['PerformingPhysicianName', 'OperatorName']

#kontrollera att ändringen blev bra
#print(names.columns)

Index(['id', 'AcquisitionPlaneId', 'DoseAreaProductTotal', 'DoseRPTotal',
       'FluoroDoseAreaProductTotal', 'FluoroDoseRPTotal', 'TotalFluoroTime',
       'AcquisitionDoseAreaProductTotal', 'AcquisitionDoseRPTotal',
       'TotalAcquisitionTime',
       ...
       'AETitle', 'DicomPort', 'InventorySystemId', 'AutoQAModalityModelId',
       'AutoQASpecificMachineSettings', 'ActiveAutoQA', 'ActiveUpdateAutoQa',
       'ModelName', 'MachineTypeId', 'ActiveREMBox'],
      dtype='object', length=163)
Index(['OperatorPseudonym', ' OperatorName'], dtype='object')
Index(['PerformingPhysicianName', 'OperatorName'], dtype='object')


In [47]:
print(data_names_S08.SourceOfDoseInformation.unique())

['Image']


In [4]:
#Joina dataframes för att få in operatörsnamn
data_names = pd.merge(data,names,how = 'left', on = ['PerformingPhysicianName'])

#Print för att kolla så att det funkade
#print(data_names.OperatorName)

In [5]:
#Kontroll av antal rader (irradiation events)
len(data)

15406

In [6]:
# Konvertera till rätt datumformat

data['DateTimeStarted'] = pd.to_datetime(data['DateTimeStarted'],infer_datetime_format=True)

#Kontroll så det funkade
#data['DateTimeStarted']

In [33]:
#Kolla vilka lab som är med i datat samt antal irradiation events
Labs = data_names.DisplayName.unique()
print(Labs)
print(len(data_names))
#Välj endast Skellefteå S08
data_names_S08 = data_names[data_names.DisplayName == 'S08']
Labs2 = data_names_S08.DisplayName.unique()
print(Labs2)
print(len(data_names_S08))

['U110' 'L3' 'S08' 'U208']
15406
['S08']
524


In [35]:
# Kontrollera specifika kolumner i Data
operators_skel = data_names_S08.OperatorName.unique()
print(operators_skel)

[' Malte Granten' ' Louise Berglund' ' Erik Lindberg' ' Hampus Studahl'
 ' Per Öberg' ' Malte Granten\\Malte Granten' ' Andreas Larsson'
 ' Peter Sundström' ' Lena Lindgren' nan ' Gataa Khaldun Ghali'
 ' Annika Öhman' ' Leif Carlgren' ' Athanasios Zouzos' ' Hampus'
 ' Lena Eriksson']


In [36]:
# Korrigera dubbletter
data_names_S08_clean = data_names_S08.replace({'OperatorName': {' Malte Granten\\Malte Granten': ' Malte Granten',
                                        ' Hampus': ' Hampus Studahl'}})

operators_skel_clean = data_names_S08_clean.OperatorName.unique()
print(operators_skel_clean)

[' Malte Granten' ' Louise Berglund' ' Erik Lindberg' ' Hampus Studahl'
 ' Per Öberg' ' Andreas Larsson' ' Peter Sundström' ' Lena Lindgren' nan
 ' Gataa Khaldun Ghali' ' Annika Öhman' ' Leif Carlgren'
 ' Athanasios Zouzos' ' Lena Eriksson']


In [41]:
#Kopiera data och behåll bara en rad per studie
data_studies_S08 = data_names_S08_clean.copy()
data_studies_S08.drop_duplicates(subset ="AccessionNumber", keep = 'first', inplace = True)

#Kontroll av antal rader som kvarstår
print(len(data_studies_S08))
print(len(data_names_S08_clean))

60
524


In [37]:
series_S08 = data_names_S08_clean[["AccessionNumber","id","StudyDateTime","DoseAreaProductTotal","DoseRPTotal",
                                "FluoroDoseAreaProductTotal","FluoroDoseRPTotal","TotalFluoroTime",
                                "AcquisitionDoseAreaProductTotal","AcquisitionDoseRPTotal","TotalAcquisitionTime",
                                "PatientAge","StudyDescription","PerformingPhysicianName","OperatorName",
                                "RequestedProcedureCode","DisplayName"]]
series_S08.head()

#NÅTT SKUMT HÄR EFTERSOM VI BARA HAR NULL I RP! FELSÖK!
#TA ÄVEN MED FLER KOLUMNER OM DETTA SKA VARA PÅ SERIENIVÅ ISTÄLLET FÖR STUDYNIVÅ

,AccessionNumber,id,StudyDateTime,DoseAreaProductTotal,DoseRPTotal,FluoroDoseAreaProductTotal,FluoroDoseRPTotal,TotalFluoroTime,AcquisitionDoseAreaProductTotal,AcquisitionDoseRPTotal,TotalAcquisitionTime,PatientAge,StudyDescription,PerformingPhysicianName,OperatorName,RequestedProcedureCode,DisplayName
1635,SERSKE0007284571,40,2020-12-10T06:59:47,1.5945,NaN,1.5945,NaN,2382.0,NaN,NaN,NaN,76,ESOFAGUS BARIUM,PN_TgWnHFtSwu2uGK1Bc5KrVHBLXa5TLkDa3LLRaqJDgQE=,Malte Granten,41100.0,S08
1636,SERSKE0007284571,40,2020-12-10T06:59:47,1.5945,NaN,1.5945,NaN,2382.0,NaN,NaN,NaN,76,ESOFAGUS BARIUM,PN_TgWnHFtSwu2uGK1Bc5KrVHBLXa5TLkDa3LLRaqJDgQE=,Malte Granten,41100.0,S08
1637,SERSKE0007284571,40,2020-12-10T06:59:47,1.5945,NaN,1.5945,NaN,2382.0,NaN,NaN,NaN,76,ESOFAGUS BARIUM,PN_TgWnHFtSwu2uGK1Bc5KrVHBLXa5TLkDa3LLRaqJDgQE=,Malte Granten,41100.0,S08
1638,SERSKE0007284571,40,2020-12-10T06:59:47,1.5945,NaN,1.5945,NaN,2382.0,NaN,NaN,NaN,76,ESOFAGUS BARIUM,PN_TgWnHFtSwu2uGK1Bc5KrVHBLXa5TLkDa3LLRaqJDgQE=,Malte Granten,41100.0,S08
1639,SERSKE0007284571,40,2020-12-10T06:59:47,1.5945,NaN,1.5945,NaN,2382.0,NaN,NaN,NaN,76,ESOFAGUS BARIUM,PN_TgWnHFtSwu2uGK1Bc5KrVHBLXa5TLkDa3LLRaqJDgQE=,Malte Granten,41100.0,S08


In [38]:
#Skriv ut dataframe till excel. För att få överblick.
import xlwt
import openpyxl
import xlsxwriter

writer = pd.ExcelWriter('CG_data/S08.xlsx', engine='xlsxwriter')
series_S08.to_excel(writer, index=False)
writer.save()

In [40]:
# Lista och visa alla använda protokoll
unikprot = data_names_S08_clean.AcquisitionProtocol.unique()

data_protokoll = pd.DataFrame(columns=['Protokoll', 'Events'])
i=0
for i in range(len(unikprot)):
    data_protokoll = data_protokoll.append({'Protokoll': unikprot[i], 
                                            'Events': len(data_names_S08_clean[data_names_S08_clean['AcquisitionProtocol']==unikprot[i]])}, 
                                           ignore_index=True)

fig = px.pie(data_protokoll, values='Events', names='Protokoll', title='Fördelning av protokoll')
fig.show()
print(len(data_names_S08_clean))

524


In [48]:
#Box plot av DAP och Gml-tid vs operatör

fig = go.Figure()

i=0
for i in range(len(operators_skel_clean)):
    fig.add_trace(
        go.Bar(x=data_studies_S08[data_studies_S08['OperatorName']==operators_skel_clean[i]].OperatorName,
               y=data_studies_S08[data_studies_S08['OperatorName']==operators_skel_clean[i]].TotalFluoroTime,
               offsetgroup=0+i
              )
    )
    
fig.update_layout(title="Genomlysningstid vs Operatör",
                  yaxis_title="Genomlysningstid"
                 )


fig.show()

In [45]:
#Basic Plot

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=data_studies_S08['DateTimeStarted'],
               y=data_studies_S08['TotalFluoroTime'],
               mode='markers')
)

fig.update_layout(title='DAP vs event',
                  yaxis_title='DAP', xaxis_title='Event'
)

fig.show()

tot = len(data_names_S08)

print(f'Totalt {tot} pedaltramp!')

Totalt 524 pedaltramp!


In [8]:
data_vv = data.dropna(subset=['PatientEquivalentThickness'])

A = len(data)
B = len(data_vv)
unikprot_vv = data_vv.AcquisitionProtocol.unique()
C = len(unikprot_vv)
D = len(unikprot)

print(f'Totalt {A} varav {B} har vattenvärden och är fördelade på {C} protokoll jämfört med {D} ursprungliga protokoll')
unikprot_vv

Totalt 3701 varav 980 har vattenvärden och är fördelade på 18 protokoll jämfört med 23 ursprungliga protokoll


array(['FL Låg Esofagus Bari', 'Esofagus Bari Gml Norm',
       'FL Låg Esofagus Jod', 'FL Norm Barn Esof Bari',
       'FL Hög Barn Esof Jod', 'FL Norm Esofagus Bari', 'Esofagus Barium',
       'Esofagus Jod Gml', 'FL Normal Esofagus Jod',
       'FL Hög Esofagus Jod', 'FL Låg Esofagus Barium',
       'FL Normal Esofagus Bar', 'Thorax Singel', 'FL Låg Thorax',
       'Esofagus Barium Singel', 'FL Låg Barn Esof Jod',
       'Esofagus Jod Singel', 'FL Norm Esofagus Jod'], dtype=object)

In [94]:
# Beroenden av vattenvärdet PatientEquivalentThickness

#Konvertera L3 vattenvärden med faktor 10
#j=0
#for j in range(len(data_vv)):
#    if data_vv['DisplayName'][j] == 'L3':
#        data_vv['PatientEquivalentThickness'][j] = data_vv['PatientEquivalentThickness'][j]/10

#Droppa L3 pga faktor 10 fel på vv
#data_vv = data_vv[data_vv['DisplayName'] != 'L3']

fig = go.Figure()

i=0
for i in range(len(unikprot_vv)):
    fig.add_trace(
        go.Scatter(x=data_vv[data_vv['AcquisitionProtocol']==unikprot_vv[i]].PatientEquivalentThickness,
                   y=data_vv[data_vv['AcquisitionProtocol']==unikprot_vv[i]].DisplayName, #DoseRP,
                   mode='markers',
                   name=unikprot_vv[i])
    )


fig.update_layout(title='Lab vs WV',
                  yaxis_title='Lab', xaxis_title='water value [cm]'
)

#fig.update_xaxes(title_text='Event', row=1, col=1)    
#fig.update_xaxes(title_text='Event', row=1, col=2)
#fig.update_xaxes(title_text='Event', row=1, col=3)

#tot_fl = len(data_vv[data_vv['AcquisitionProtocol']=='FL Norm Esofagus Bari'])
#tot_exp = len(data_vv[data_vv['AcquisitionProtocol']=='Esofagus Barium'])

#print(f'Totalt {tot_fl} genomlysningsevents och {tot_exp} exponeringsevents')

fig.show()


In [16]:
# Esofagus/Hypofarynx för 6 olika patienter

#Gör 6 subplots!
fig = make_subplots(rows=2, cols=3,
                    specs=[[{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}]],
                    subplot_titles=("Patient 1", "Patient 2", "Patient 3", "Patient 4", "Patient 5", "Patient 6"),
                    x_title = 'Irradiation Events',
                    y_title = 'Patient Thickness',
                    vertical_spacing=0.1,
                    horizontal_spacing=0.1                    
                   )


#Sortera fram en lista på unika Accessionnumbers för en agiven modalitet, 
unikacc_vv = data_vv[data_vv.DisplayName=='U110'].AccessionNumber.unique()

#Loopa över de 6 sista i listan av unika accessionnumbers. 
#Fördela protokollnamnen på olika legends. Plotta event på x och vattenvärde på y
#Nya traces med kV, mA, pulsbredd, Cu i samma graf. 
#Skulle vara intressant med pie-charts för varje undersökning med fördelning av DoseRP vs event. Hur stor del är positionering?
# ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']


rad = [1,1,1,2,2,2]
kol = [1,2,3,1,2,3]
i=0
for i in range(6):
    fig.add_trace(
        go.Scatter(x=data_vv[data_vv.AccessionNumber==unikacc_vv[i+25]].DateTimeStarted,
                   y=data_vv[data_vv.AccessionNumber==unikacc_vv[i+25]].PatientEquivalentThickness,
                   mode='markers',
                   name='PatientThickness',
                   line = dict(color='#FECB52')
                  ),
        row=rad[i], col=kol[i], secondary_y=False,
    )
    fig.add_trace(
            go.Scatter(x=data_vv[data_vv.AccessionNumber==unikacc_vv[i+25]].DateTimeStarted,
                       y=data_vv[data_vv.AccessionNumber==unikacc_vv[i+25]].kVp,
                       mode='markers',
                       name='kVp',
                       line = dict(color='#636EFA')
                      ),
        row=rad[i], col=kol[i], #secondary_y=True,
    )



    
fig.update_layout(title='WV vs Event',
                  legend={'traceorder':'reversed'}
)

fig.update_yaxes(range=[0, 120])
fig.update_yaxes(nticks=20, dtick=20)
fig.update_xaxes(nticks=10)
fig.update_xaxes(showticklabels=False)


#fig.update_xaxes(title_text='Event', row=1, col=1)    
#fig.update_xaxes(title_text='Event', row=1, col=2)
#fig.update_xaxes(title_text='Event', row=1, col=3)

#tot_fl = len(data_vv[data_vv['AcquisitionProtocol']=='FL Norm Esofagus Bari'])
#tot_exp = len(data_vv[data_vv['AcquisitionProtocol']=='Esofagus Barium'])

#print(f'Totalt {tot_fl} genomlysningsevents och {tot_exp} exponeringsevents')

fig.show()

In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].DateTimeStarted,
    y=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].PatientEquivalentThickness,
    mode='markers',
    name='PatientThickness'#,
    #line = dict(color='firebrick')
             ))

fig.add_trace(go.Scatter(
    x=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].DateTimeStarted,
    y=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].kVp,
    mode='markers',
    name='kVp',
    yaxis="y2"#,
    #line = dict(color='royalblue')
             ))

fig.add_trace(go.Scatter(
    x=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].DateTimeStarted,
    y=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].XrayTubeCurrent,
    mode='markers',
    name='mA',
    yaxis="y3"#,
    #line = dict(color='orange')
             ))

fig.update_layout(
    xaxis=dict(
        domain=[0.3, 0.7]
    ),
    yaxis=dict(
        title="Patient Thickness [cm]",
        titlefont=dict(color='#636EFA'),
        tickfont=dict(color='#636EFA')
    ),
    yaxis2=dict(
        title="kVp",
        titlefont=dict(color='#EF553B'),
        tickfont=dict(color='#EF553B'),
        anchor="x",
        overlaying="y",
        side="right",
        position=0.85
    ),
    yaxis3=dict(
        title="mA",
        titlefont=dict(color='#00CC96'),
        tickfont=dict(color='#00CC96'),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.8

    )
)


fig.show()


In [23]:
fig = make_subplots(rows=5, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.02,
                    row_titles=['kV','mA','ms','Thickness','Cu']
                   )
# ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']

fig.add_trace(go.Scatter(
    x=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].DateTimeStarted,
    y=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].kVp,
    mode='markers',
    name='kVp',
    line = dict(color='#636EFA')),
              row=1, col=1
             )

fig.add_trace(go.Scatter(
    x=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].DateTimeStarted,
    y=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].XrayTubeCurrent,
    mode='markers',
    name='mA',
    line = dict(color='#EF553B')),
              row=2, col=1
             )

fig.add_trace(go.Scatter(
    x=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].DateTimeStarted,
    y=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].PulseWidth*1000,
    mode='markers',
    name='ms',
    line = dict(color='#00CC96')),
              row=3, col=1
             )

fig.add_trace(go.Scatter(
    x=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].DateTimeStarted,
    y=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].PatientEquivalentThickness,
    mode='markers',
    name='PatientThickness',
    line = dict(color='#FECB52')),
             row=4, col=1
             )

fig.add_trace(go.Scatter(
    x=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].DateTimeStarted,
    y=data_vv[data_vv.AccessionNumber==unikacc_vv[28]].XrayFilterThicknessMinimum,
    mode='markers',
    name='Cu',
    line = dict(color='#FFA15A')),
             row=5, col=1
             )

fig.update_yaxes(tickvals=[0.1, 0.3, 0.6, 0.9], row=5)

fig.update_layout(
    autosize=False,
    width=1100,
    height=600,
    showlegend=False
)


fig.show()


In [81]:
df2 = pd.DataFrame(np.array([['L3', 300, 70], ['U110', 35, 72], ['S08', 50, 79], ['L3', 350, 72]]), columns=['Lab', 'pt', 'kV'])

df2['pt'] = df2['pt'].astype(int)

i=0
for i in range(len(df2)):
    if df2['Lab'][i] == 'L3':
        df2['pt'][i] = df2['pt'][i]/10

#j=0
#for j in range(len(data_vv)):
#    if data_vv['DisplayName'][j] == 'L3':
#        data_vv['PatientEquivalentThickness'][j] = data_vv['PatientEquivalentThickness'][j]/10

#conditions = [
#    (df2['Lab'] == 'L3'),
#    (df2['Lab'] == 'U110'),
#    (df2['Lab'] == 'S08')]
#choices = [10,1,1]

#df2['vikt'] = np.select(conditions, choices, default='null')
#df2['pt'] = df2['pt'].astype(int)
#df2['vikt'] = df2['vikt'].astype(int)
#df2['ny_pt'] = df2['pt']/df2['vikt']

df2

,Lab,pt,kV
0,L3,30,70
1,U110,35,72
2,S08,50,79
3,L3,35,72
